In [1]:
import pandas as pd
import numpy as np

In [2]:
## Get Target columns from csv file
df_target = pd.read_csv("train/train.csv",usecols = range(1933,1934))

In [3]:
df_target.head()

,target
0,0
1,0
2,0
3,0
4,1


In [4]:
## Get first 10 data columns
df=pd.read_csv("train/train.csv",usecols=range(1,10))

C:\Users\Brian\Miniconda2\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
## Show first row values (Transposed)
df.head().T

,0,1,2,3,4
VAR_0001,H,H,H,H,R
VAR_0002,224,7,116,240,72
VAR_0003,0,53,3,300,261
VAR_0004,4300,4448,3464,3200,2000
VAR_0005,C,B,C,C,N
VAR_0006,0,1,0,0,0
VAR_0007,0,0,0,0,0
VAR_0008,False,False,False,False,False
VAR_0009,False,False,False,False,False


In [6]:
## Add target column to data frame
df = pd.concat([df, df_target], axis=1, join_axes=[df.index])

In [7]:
## Examine datatypes - will need to convert objects to numeric
df.dtypes

VAR_0001     object
VAR_0002      int64
VAR_0003      int64
VAR_0004      int64
VAR_0005     object
VAR_0006    float64
VAR_0007    float64
VAR_0008     object
VAR_0009     object
target        int64
dtype: object

In [13]:
## Check for the number of unique values for each of the 'object' columns

max_unique_vals =2  # Set maximum unique values for 'object' columns
    # We don't want to create too many normalized columns
df_ohe_list =[]  # Prepare list for one-hot-encoding
df_drop_list=[]  # Prepare list for columns to drop
for i in df.columns.values:
    if df[i].dtype == 'object':
        print i, len(df[i].unique())
        if (len(df[i].unique())) > max_unique_vals:
            df_drop_list.append(i)
        else:
            df_ohe_list.append(i)

VAR_0001 3
VAR_0005 4
VAR_0008 2
VAR_0009 2


In [14]:
df_drop_list

['VAR_0001', 'VAR_0005']

In [20]:
## Drop dataframe columns with two many categorical values
df=df.drop(df_drop_list,axis=1)

In [15]:
## Apply One-Hot Encoding to the 'object' columns
df=pd.get_dummies(df,columns=df_ohe_list)

In [21]:
## Examine data frame
df.head().T

,0,1,2,3,4
VAR_0002,224.0,7.0,116.0,240.0,72.0
VAR_0003,0.0,53.0,3.0,300.0,261.0
VAR_0004,4300.0,4448.0,3464.0,3200.0,2000.0
VAR_0006,0.0,1.0,0.0,0.0,0.0
VAR_0007,0.0,0.0,0.0,0.0,0.0
target,0.0,0.0,0.0,0.0,1.0
VAR_0008_False,1.0,1.0,1.0,1.0,1.0
VAR_0009_False,1.0,1.0,1.0,1.0,1.0


In [25]:
## Identify columns with outliers
for i in df.columns:
    print df[i].describe(percentiles = [.02,.05,.25, .5, .75,.95,.98])

count    132253.000000
mean         90.351221
std          97.332864
min           0.000000
2%            2.000000
5%            4.000000
25%          23.000000
50%          60.000000
75%         125.000000
95%         300.000000
98%         393.000000
max         523.000000
Name: VAR_0002, dtype: float64
count    132253.000000
mean         75.460042
std          78.662328
min           0.000000
2%            0.000000
5%            0.000000
25%          13.000000
50%          53.000000
75%         112.000000
95%         244.000000
98%         303.000000
max         385.000000
Name: VAR_0003, dtype: float64
count    132253.000000
mean       2810.042419
std        1568.609275
min           0.000000
2%            0.000000
5%          961.000000
25%        1761.000000
50%        2500.000000
75%        3500.000000
95%        5800.000000
98%        7200.000000
max       11421.000000
Name: VAR_0004, dtype: float64
count    132253.000000
mean          0.500087
std           0.950166
min       

In [23]:
## Handling Outliers - remove any values outside of the 98 percentile
## Statistically there is probably a better way to handle this, open for suggestions.

## Show any rows beyond the 98 percentile

for i in df.columns:
    print i,df[i].quantile(.98)
    print df[df[i]<=df[i].quantile(.98)].count().max()
    df = df[df[i]<=df[i].quantile(.98)]

VAR_0002 523.0
142330
VAR_0003 385.0
139496
VAR_0004 11421.4
136706
VAR_0006 5.0
133970
VAR_0007 3.0
132253
target 1.0
132253
VAR_0008_False 1.0
132253
VAR_0009_False 1.0
132253


In [26]:
## Identify columns with nan and replace with the mean for that column
for i in df.columns:
    if (pd.isnull(df[i]).sum() > 0):
        mean = df[i].mean()
        df[i]=df[i].fillna(mean)

In [28]:
## Min max scaler
for i in df.columns:
    print i
    col_min = df[i].min()
    print col_min
    col_max = df[i].max()
    print col_max
    col_range = col_max-col_min
    print col_range
    if col_range>0:
        df[i]=(df[i]-col_min)/col_range
    

VAR_0002
0
523
523
VAR_0003
0
385
385
VAR_0004
0
11421
11421
VAR_0006
0.0
5.0
5.0
VAR_0007
0.0
3.0
3.0
target
0
1
1
VAR_0008_False
1
1
0
VAR_0009_False
1
1
0


In [29]:
df.head().T

,0,1,2,3,4
VAR_0002,0.428298,0.013384,0.221797,0.458891,0.137667
VAR_0003,0.000000,0.137662,0.007792,0.779221,0.677922
VAR_0004,0.376499,0.389458,0.303301,0.280186,0.175116
VAR_0006,0.000000,0.200000,0.000000,0.000000,0.000000
VAR_0007,0.000000,0.000000,0.000000,0.000000,0.000000
target,0.000000,0.000000,0.000000,0.000000,1.000000
VAR_0008_False,1.000000,1.000000,1.000000,1.000000,1.000000
VAR_0009_False,1.000000,1.000000,1.000000,1.000000,1.000000


In [27]:
## Another method of doing min / max scaling using sklearn preprocessing

from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()
np_scaled = min_max_scaler.fit_transform(df)
type(np_scaled)

df_normalized = pd.DataFrame(np_scaled,columns=df.columns)
df_normalized.head().T


,0,1,2,3,4
VAR_0002,0.428298,0.013384,0.221797,0.458891,0.137667
VAR_0003,0.000000,0.137662,0.007792,0.779221,0.677922
VAR_0004,0.376499,0.389458,0.303301,0.280186,0.175116
VAR_0006,0.000000,0.200000,0.000000,0.000000,0.000000
VAR_0007,0.000000,0.000000,0.000000,0.000000,0.000000
target,0.000000,0.000000,0.000000,0.000000,1.000000
VAR_0008_False,0.000000,0.000000,0.000000,0.000000,0.000000
VAR_0009_False,0.000000,0.000000,0.000000,0.000000,0.000000


In [32]:
# Import train_test_split
from sklearn.cross_validation import train_test_split

# Split the 'features' and 'income' data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df, df.target, test_size = 0.2, random_state = 0)

# Show the results of the split
print "Training set has {} samples.".format(X_train.shape[0])
print "Testing set has {} samples.".format(X_test.shape[0])

Training set has 105802 samples.
Testing set has 26451 samples.


In [33]:
## Make a naive prediction that all target values are zero

## To get the number of zero values, it will be the length of the vector minus the sum of the values
## Then divide by the length of the vector to get the accuracy
accuracy = float(len(y_train)-sum(y_train))/len(y_train)

print accuracy

## So our model would have to improve on this accuracy

0.775722576133


In [43]:
# TODO: Import two metrics from sklearn - fbeta_score and accuracy_score
from sklearn.metrics import accuracy_score, fbeta_score

from time import time

def train_predict(learner, sample_size, X_train, y_train, X_test, y_test): 
    '''
    inputs:
       - learner: the learning algorithm to be trained and predicted on
       - sample_size: the size of samples (number) to be drawn from training set
       - X_train: features training set
       - y_train: income training set
       - X_test: features testing set
       - y_test: income testing set
    '''    
    results = {}
    
    # TODO: Fit the learner to the training data using slicing with 'sample_size'
    start = time() # Get start time
    learner.fit(X_train[:sample_size],y_train[:sample_size])
    end = time() # Get end time
    print "Type of X_train is %s " % type(X_train)
    print "Shape of X_train is %s " % X_train.shape[0]
    print "Shape of y_train is %s " % y_train[:sample_size].shape[0]
    
    # TODO: Calculate the training time
    results['train_time'] = end-start
    print "Train time is %s " % results['train_time']
        
    # TODO: Get the predictions on the test set,
    #       then get predictions on the first 300 training samples
    start = time() # Get start time
    #predictions_test = learner.predict(X_test[:300])
    predictions_test = learner.predict(X_test)
    predictions_train = learner.predict(X_train)
    end = time() # Get end time
    
    # TODO: Calculate the total prediction time
    results['pred_time'] = end-start
    print "Pred time is %s " % results['pred_time']
            
    # TODO: Compute accuracy on the first 300 training samples
    results['acc_train'] = accuracy_score(predictions_train, y_train)
        
    # TODO: Compute accuracy on test set
    results['acc_test'] = accuracy_score(predictions_test, y_test)
    
    # TODO: Compute F-score on the the first 300 training samples
    #results['f_train'] = fbeta_score(predictions_train, y_train[:300], average=None, beta=0.5)
    results['f_train'] = fbeta_score(predictions_train, y_train, beta=0.5)
        
    # TODO: Compute F-score on the test set
    #results['f_test'] = fbeta_score(predictions_test, y_test[:300], average=None, beta=0.5)
    results['f_test'] = fbeta_score(predictions_test, y_test, beta=0.5)
       
    # Success
    print "{} trained on {} samples.".format(learner.__class__.__name__, sample_size)
        
    # Return the results
    return results

In [45]:
log_reg_lrnr = LogisticRegression()

In [47]:
log_reg_lrnr.fit(X_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [48]:
predictions_test = log_reg_lrnr.predict(X_test)

In [51]:
test_df =X_test

In [60]:
len(test_df)

26451

In [61]:
pred_test_df = pd.DataFrame(predictions_test)

In [54]:
type(predictions_test)

numpy.ndarray

In [64]:
pred_test_df[pred_test_df.index==15998].head().T

,15998
0,0.0


In [58]:
test_df[test_df.index==117171].head().T

,117171
VAR_0002,0.116635
VAR_0003,0.376623
VAR_0004,0.218457
VAR_0006,0.000000
VAR_0007,0.000000
target,0.000000
VAR_0008_False,1.000000
VAR_0009_False,1.000000
0,NaN


In [63]:
test_df.head().T

,117171,60486,15998,105251,71188
VAR_0002,0.116635,0.007648,0.003824,0.141491,0.000000
VAR_0003,0.376623,0.618182,0.111688,0.127273,0.000000
VAR_0004,0.218457,0.263637,0.294458,0.096314,0.123107
VAR_0006,0.000000,0.200000,0.000000,0.800000,0.000000
VAR_0007,0.000000,0.333333,0.000000,0.333333,0.000000
target,0.000000,0.000000,0.000000,0.000000,1.000000
VAR_0008_False,1.000000,1.000000,1.000000,1.000000,1.000000
VAR_0009_False,1.000000,1.000000,1.000000,1.000000,1.000000
0,NaN,NaN,0.000000,NaN,NaN


In [56]:
test_df = pd.concat([test_df, pd.DataFrame(predictions_test)], axis=1, join_axes=[test_df.index])

In [44]:
# TODO: Import the three supervised learning models from sklearn
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier

# TODO: Initialize the three models
clf_A = LogisticRegression()
clf_B = LinearSVC()
clf_C = KNeighborsClassifier()

# TODO: Calculate the number of samples for 1%, 10%, and 100% of the training data
samples_1 = int(len(df) * 0.01)
samples_10 = int(len(df) * 0.1)
samples_100 = len(df)

# Collect results on the learners
results = {}
for clf in [clf_A, clf_B, clf_C]:
    clf_name = clf.__class__.__name__
    results[clf_name] = {}
    for i, samples in enumerate([samples_1, samples_10, samples_100]):
        results[clf_name][i] = \
        train_predict(clf, samples, X_train, y_train, X_test, y_test)

# Run metrics visualization for the three supervised learning models chosen
print "After train predict, before vs.evaluate"
print results
#print accuracy
#print fscore
#vs.evaluate(results, accuracy, fscore)

Type of X_train is <class 'pandas.core.frame.DataFrame'> 
Shape of X_train is 105802 
Shape of y_train is 1322 
Train time is 0.0119998455048 
Pred time is 0.156999826431 
LogisticRegression trained on 1322 samples.
Type of X_train is <class 'pandas.core.frame.DataFrame'> 
Shape of X_train is 105802 
Shape of y_train is 13225 
Train time is 0.0249998569489 
Pred time is 0.00899982452393 
LogisticRegression trained on 13225 samples.
Type of X_train is <class 'pandas.core.frame.DataFrame'> 
Shape of X_train is 105802 
Shape of y_train is 105802 
Train time is 0.268000125885 
Pred time is 0.00799989700317 
LogisticRegression trained on 132253 samples.
Type of X_train is <class 'pandas.core.frame.DataFrame'> 
Shape of X_train is 105802 
Shape of y_train is 1322 
Train time is 0.00600004196167 
Pred time is 0.00699996948242 
LinearSVC trained on 1322 samples.
Type of X_train is <class 'pandas.core.frame.DataFrame'> 
Shape of X_train is 105802 
Shape of y_train is 13225 
Train time is 0.0740

NameError: name 'fscore' is not defined